In [11]:
import os, requests, urllib, zipfile
from bs4 import BeautifulSoup as bs
import xml.etree.ElementTree as ET
from datetime import datetime as date

# root domain of German parliament website
root = 'https://www.bundestag.de'
# ajax request url
asin = '/ajax/filterlist/de/services/opendata/488214-488214?limit=10&noFilterSet=true'

# HTTP request
page = requests.get(root + asin)


In [74]:

# assign BeautifulSoup object on response content
soup = bs(page.content)

# TODO: clean the data folder first
directory = os.fsencode('./data/xml/')

# loop all anchor with this class
for a in soup.find_all(attrs={'class':'bt-link-dokument'}):
    # retrieve plenary zip file from hrefs
    zipcall, _ = urllib.request.urlretrieve(root + a['href'])
    # extract all xml files of this legislature into raw folder
    with zipfile.ZipFile(zipcall, "r") as f:
        f.extractall(directory)



In [42]:
import xml.etree.ElementTree as ET
from datetime import datetime as date

directory = './data/xml/'

for file in os.listdir(os.fsencode(directory)):
    filename = os.fsdecode(file)
    if filename.endswith('.xml'):
        root = ET.parse(directory + filename).getroot()
        for child in root.iter('DATUM'):
            plenaryDate = date.strptime(child.text, '%d.%m.%Y').strftime('%Y-%m-%d')
            os.rename(directory + filename,  directory + plenaryDate + '.xml')
    else: 
        os.remove(directory + filename)


